In [9]:
# Import Libraries
import pandas as pd                                       # For Preprocessing
import numpy as np
import tensorflow as tf
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow_hub as tf_hub
from tensorflow.keras.models import load_model
import string

In [10]:
# Load New Data
df_inf = pd.read_csv('new_data.csv')
df_result = df_inf.copy()
df_inf_list = df_inf['headline'].tolist()

In [11]:
# Create A Function for Text Preprocessing
def text_preprocessing(text):
  # Case folding
  text = text.lower()

  # Newline removal (\n)
  text = re.sub(r"\\n", " ",text)

  # Whitespace removal
  text = text.strip()

  # Puntuaction
  text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub(r'\d+', '', text)

  # Non-letter removal (such as emoticon, symbol (like μ, $, 兀), etc
  text = re.sub("[^A-Za-z\s']", " ", text)

  # Tokenization
  tokens = word_tokenize(text)

  # Define Stopwords
  stpwds_id = list(set(stopwords.words('english')))

  # Stopwords removal
  tokens = [word for word in tokens if word not in stpwds_id]

  # Lemmatizer  
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  # Combining Tokens
  text = ' '.join(tokens)
  return text

# Preprocessing layer
def preprocessing_layer(text_tensor):
    def _inner_py_func(text_b):
        text = text_b.decode('utf-8')
        return text_preprocessing(text)
    
    result = tf.numpy_function(_inner_py_func, [text_tensor], tf.string)
    return result

In [12]:
# Multiple Inferences
model_path = 'deployment/final_model.keras'
model = tf.keras.models.load_model(
    model_path, 
    custom_objects={
        'preprocessing_layer': preprocessing_layer,
        'KerasLayer': tf_hub.KerasLayer
    },
    safe_mode=False
)

predictions = model.predict(np.array(df_inf_list))
target_indices = np.argmax(predictions, axis=1)

labels = ['Negative', 'Neutral', 'Positive'] 

predicted_labels = [labels[idx] for idx in target_indices]

df_result = pd.DataFrame({
    'text': df_inf_list,
    'label': predicted_labels
})

df_result

108/108 [==============================] - 4s 32ms/step


,text,label
0,Apple and Google agree app store changes to ap...,Positive
1,How Investors Are Reacting To Apple (AAPL) Q1 ...,Positive
2,TSMC Stock Is on a Roll as Chip Sales Surge. T...,Neutral
3,Bull of the Day: Broadcom (AVGO) Discover why ...,Positive
4,1 Unstoppable Stock To Buy Before It Joins Nvi...,Neutral
...,...,...
3420,106-year-old retailer closing all stores in Ch...,Neutral
3421,"If You'd Invested $100 in Costco 10 Years Ago,...",Neutral
3422,Walmart’s (WMT) Technology Push Draws Optimism...,Neutral
3423,The DGI Core Portfolio: 2025 Year-End Review,Neutral


In [13]:
# Multiple Inference Result Summary
df_result['label'].value_counts()

label
Neutral     2439
Positive     655
Negative     331
Name: count, dtype: int64

In [15]:
# Single Inference Result
# New Data
sample = "Why Google stock is shelved in 2026."

# Text Preprocessing
sample_processed = text_preprocessing(sample)

# Predict
result = model.predict(tf.constant([sample_processed]))

# Transfrom to Target Label
higher_value = np.argmax(result[0])
sentiment_result = labels[higher_value]
print(f"Prediction: {sentiment_result}")

1/1 [==============================] - 0s 26ms/step
Prediction: Negative
